In [47]:
"""
Testing a 10 month SMA strategy using month end data. The strategy is to sell when the S&P500 goes below the 10 month
SMA and buy when the S&P500 rises above the 10 month SMA. Buy and sell signals are only made at month end. 

Month-end data from January 1950 was downloaded from Yahoo Finance.


"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

pd.set_option("display.max_rows", None, "display.max_columns", None)

%matplotlib inline

In [48]:
#Read in month end data from 1950 from csv

csvpath = Path(f"C:/Users/costa/Desktop/byte/rice/ru-hou-fin-pt-03-2020-u-c/Project2_draft/SPY_month_end.csv",
                  index_col="date", infer_datetime_format=True, parse_dates=True)
stock_data_df = pd.read_csv(csvpath)
#stock_data_df['Date'] = pd.to_datetime(stock_data_df['Date'] )
stock_data_df.set_index('Date',inplace=True) 
stock_data_df.head()

,Monthly Close
Date,
01/03/50,17.05
02/01/50,17.22
03/01/50,17.29
04/03/50,17.96
05/01/50,18.78


In [49]:
#Clean data

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,Monthly Close
Date,
01/03/50,17.05
02/01/50,17.22
03/01/50,17.29
04/03/50,17.96
05/01/50,18.78


In [51]:
# To confirm the above results (short = 1, long = 10) we will do our own tests. 
signals_df = stock_data_df['Monthly Close'].to_frame()

signals_df.head()


,Monthly Close
Date,
01/03/50,17.05
02/01/50,17.22
03/01/50,17.29
04/03/50,17.96
05/01/50,18.78


In [53]:
# Set the short window and long windows, use 1 for current price, 10 for 10 month ends
short_window = 1
long_window = 10

# Set the `date` column as the index
#signals_df.set_index("date", drop=True)

# Generate the short and long moving averages (1 and 10 days, respectively)
signals_df["SMA1"] = signals_df["Monthly Close"].rolling(window=short_window).mean()
signals_df["SMA10"] = signals_df["Monthly Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA1"][short_window:] > signals_df["SMA10"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(100)

,Monthly Close,SMA1,SMA10,Signal,Entry/Exit
Date,,,,,
01/03/50,17.05,17.05,NaN,0.0,NaN
02/01/50,17.22,17.22,NaN,0.0,0.0
03/01/50,17.29,17.29,NaN,0.0,0.0
04/03/50,17.96,17.96,NaN,0.0,0.0
05/01/50,18.78,18.78,NaN,0.0,0.0
06/01/50,17.69,17.69,NaN,0.0,0.0
07/03/50,17.84,17.84,NaN,0.0,0.0
08/01/50,18.42,18.42,NaN,0.0,0.0
09/01/50,19.45,19.45,NaN,0.0,0.0


In [55]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Monthly Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Monthly Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Monthly Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA1', 'SMA10']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                  :Curve   [Date]   (Monthly Close)
   .NdOverlay.I              :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Monthly_Close.I  :Scatter   [Date]   (Monthly Close)
   .Scatter.Monthly_Close.II :Scatter   [Date]   (Monthly Close)

In [72]:
# Set initial capital
initial_capital = float(100000)

signals_df["Portfolio Cash"] = 100000

# Set the share size
share_size = 400

# Take a position where the dual moving average crossover is 1 (short SMA is greater than long SMA)
signals_df['Position'] = share_size * signals_df['Signal']

#Find the points in time where a share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Monthly Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = signals_df['Portfolio Cash'] - (signals_df['Monthly Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# # Print the DataFrame
# pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.head(1000)



,Monthly Close,SMA1,SMA10,Signal,Entry/Exit,Portfolio Cash,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
01/03/50,17.05,17.05,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
02/01/50,17.22,17.22,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,NaN,NaN
03/01/50,17.29,17.29,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.000000,0.00000
04/03/50,17.96,17.96,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.000000,0.00000
05/01/50,18.78,18.78,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.000000,0.00000
06/01/50,17.69,17.69,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.000000,0.00000
07/03/50,17.84,17.84,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.000000,0.00000
08/01/50,18.42,18.42,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.000000,0.00000
09/01/50,19.45,19.45,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.000000,0.00000


In [73]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [74]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [75]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df



,Backtest
Annual Return,NaN
Cumulative Returns,10.5177
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [76]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annual Return,0.789522
Cumulative Returns,10.5177
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [77]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.789522
Cumulative Returns,10.5177
Annual Volatility,0.303491
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [78]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.789522
Cumulative Returns,10.5177
Annual Volatility,0.303491
Sharpe Ratio,2.60147
Sortino Ratio,NaN


In [79]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

,Backtest
Annual Return,0.789522
Cumulative Returns,10.5177
Annual Volatility,0.303491
Sharpe Ratio,2.60147
Sortino Ratio,NaN


In [80]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annual Return,0.789522
Cumulative Returns,10.5177
Annual Volatility,0.303491
Sharpe Ratio,2.60147
Sortino Ratio,3.94625


In [90]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss',
        'Percent Chg'
    ]
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss,Percent Chg


In [91]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Monthly Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Monthly Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Monthly Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
      
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Percent Chg': ((exit_portfolio_holding - entry_portfolio_holding)/entry_portfolio_holding)*100,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss,Percent Chg
0,SPY,10/02/50,03/02/53,400.0,19.53,25.29,7812.0,10116.0,2304.0,29.493088
1,SPY,01/04/54,09/04/56,400.0,26.08,45.35,10432.0,18140.0,7708.0,73.888037
2,SPY,05/01/57,08/01/57,400.0,47.43,45.22,18972.0,18088.0,-884.0,-4.659498
3,SPY,04/01/58,09/01/59,400.0,43.44,56.88,17376.0,22752.0,5376.0,30.939227
4,SPY,11/02/59,01/04/60,400.0,58.28,55.61,23312.0,22244.0,-1068.0,-4.581332
5,SPY,06/01/60,07/01/60,400.0,56.92,55.51,22768.0,22204.0,-564.0,-2.477161
6,SPY,08/01/60,09/01/60,400.0,56.96,53.52,22784.0,21408.0,-1376.0,-6.039326
7,SPY,11/01/60,04/02/62,400.0,55.54,65.24,22216.0,26096.0,3880.0,17.464890
8,SPY,11/01/62,06/01/65,400.0,62.26,84.12,24904.0,33648.0,8744.0,35.110826
9,SPY,08/02/65,03/01/66,400.0,87.17,89.23,34868.0,35692.0,824.0,2.363198


In [93]:
#Create plots

price_df = signals_df[['Monthly Close', 'SMA1', 'SMA10']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [94]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

In [32]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [33]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [34]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)